Model Setup
-----------

2D, Heat Equation with Dirichlet BC at top and bottom boundary.

\\[
\frac{\partial T}{\partial t} = -v \cdot \frac{\partial T}{\partial y} +\kappa \frac{\partial^2 T}{\partial y^2}+H
\\]

\\[
\frac{\partial T}{\partial x} = 0
\\]

with $y : [0,1]$

$T(y=1) = T_{o}$

$ \frac{\partial{T}}{\partial{y}}_{y=0} = f $

------

Effectively a 1D problem in $y$-axis, described by the analytic function

$c_{0} = ( \frac{f}{k}-\frac{h}{v} ) \cdot \frac{k}{v} \cdot \exp \left[-\frac{v}{k}y_{min} \right]$

$c_{1} = T_{o}-c_{0}\exp \left[\frac{v}{k}y_{max} \right] - \frac{h}{v}y_{max}$


$T = c_{0} \exp \left[\frac{v}{k}y\right] + \frac{h}{v}y + c_{1}$

In [ ]:
import underworld as uw
import glucifer
import numpy as np
rank = uw.rank()

# depth range
ymin = 0.0
ymax = 1.0

To = 8.0     # surface temperature
v = 1.0e-1   # vertical velocity
k = 1.0e-1   # diffusivity
h = 0.4e-1   # heat production, source term
f = -1.0     # flux = k*dT_dy


# build mesh and fields
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                 elementRes  = (10, 10), 
                                 minCoord    = (0., ymin), 
                                 maxCoord    = (1., ymax))

tField    = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1, dataType='double')
tDotField = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1, dataType='double')
vField    = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=2, dataType='double')

for ii in mesh.specialSets['MaxJ_VertexSet']:
    tField.data[ii] = To

vField.data[:] = (0.0,v)
tDotField.data[:] = 0.0

### optional visuals
# fig = glucifer.Figure()
# fig.append(glucifer.objects.Surface(mesh=mesh, fn=tField, colours=('blue white red')) )
# fig.save_image('./initial.png')


# define neumann condition
nbc = uw.conditions.NeumannCondition( flux=(0.0,f), 
                                      variable=tField, 
                                      nodeIndexSet=(mesh.specialSets['MinJ_VertexSet']) )

bc = uw.conditions.DirichletCondition(tField, indexSetsPerDof=(mesh.specialSets['MaxJ_VertexSet']) )

# define heat eq. system
ss = uw.systems.AdvectionDiffusion( phiField       = tField,
                                    phiDotField    = tDotField,
                                    velocityField  = vField,
                                    fn_diffusivity = k,
                                    fn_sourceTerm  = h,
                                    conditions     = [bc, nbc] ) 

In [ ]:
# evolve to a steady state
er = 1.0
its = 0
tOld = tField.copy()
while er > 1.0e-6:
    tOld.data[:] = tField.data[:]
    dt = ss.get_max_dt()
    ss.integrate(dt)
    
    absErr = uw.utils.nps_2norm(tOld.data-tField.data)
    magT   = uw.utils.nps_2norm(tOld.data)
    er = absErr/magT
    
    its += 1

print "Number of iterations ",its


###option visuals
# fig.save_image('./final.png')

In [ ]:
# analytic solution definitions
c0 = (f/k-h/v) * k/v * np.exp(-v/k*ymin)
c1 = To - c0*np.exp(v/k*ymax) - h/v*ymax

def exactFunc(y):
    return c0*np.exp(v/k*y) + h/v*y + c1

def exactDeriv(y):
    return v/k*c0*np.exp(v/k*y) + h/v

In [ ]:
yvals       = np.zeros(len(mesh.specialSets['MinI_VertexSet'].data))
ycoord      = np.zeros_like(yvals)
yexact_k_v1 = np.zeros_like(yvals)

ids = mesh.specialSets['MinI_VertexSet']
yvals[:] = tField.evaluate(ids).T

ycoord = tField.mesh.data[ids.data,[1]]
yexact_k_v1 = exactFunc(ycoord)

# print "The numerics: ",yvals
# print "The numerics: ",yexact_k_v1


# In[14]:

abserr = uw.utils.nps_2norm(yexact_k_v1 - yvals)
mag = uw.utils.nps_2norm(yexact_k_v1)
relerr = abserr / mag

"""
from mpi4py import MPI
comm = MPI.COMM_WORLD

# assuming order in the allgather is the same
coords = comm.allgather(ycoord)
numerical = comm.allgather(yvals)

if rank == 0:
    # build matplot lib graph of result only on proc 0

    # 1st build exact solution hiRes
    big = np.linspace(ymin,ymax)
    cool = exactFunc(big)

# %matplotlib inline
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    pyplot.plot(coords, numerical, 'o', color = 'black', label='numerical') 
    pyplot.plot(big, cool, color = 'red', label="exact") 
    pyplot.xlabel('Y coords')
    pyplot.ylabel('Temperature')
    pyplot.show()
"""

if rank == 0:
    threshold = 3.0e-4
    yspot = ymin
    print "Numerical flux at y = " ,yspot,"is", tField.fn_gradient.evaluate([0.2,yspot])[0][1]
    print "Exact flux at y=" ,yspot,"is", exactDeriv(yspot)
    print "Abs. error ", abserr, " magnitude (will change in parallel due to shadow space) ", mag
    print "Rel. error ", relerr
    if relerr > threshold:
        raise RuntimeError("The numerical solution is outside the error threshold of the analytic solution." \
                           "The Relative error was ", relerr," the threshold is ", threshold)  
        import sys
        sys.exit(1)